In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# xtrain[0] = Mushroom cap color (Brown - 1, Red - 0)
# xtrain[1] = Mushroom stalk shape (Taper - 1, Enlarge - 0)
# xtrain[2] = Mushroom Solitary (Yes - 1, No - 0)

# ytrain = (Edible - 1, Not edible - 0)

In [3]:
xtrain = np.array([[1,1,1],
                   [1,0,1],
                   [1,0,0],
                   [1,0,0],
                   [1,1,1],
                   [0,1,1],
                   [0,0,0],
                   [1,0,1],
                   [0,1,0],
                   [1,0,0]])

ytrain = np.array([1,1,0,0,1,0,0,1,1,0])

In [4]:
print(xtrain[0:10])

[[1 1 1]
 [1 0 1]
 [1 0 0]
 [1 0 0]
 [1 1 1]
 [0 1 1]
 [0 0 0]
 [1 0 1]
 [0 1 0]
 [1 0 0]]


In [5]:
print(ytrain)

[1 1 0 0 1 0 0 1 1 0]


In [6]:
print("Shape XTrain -", xtrain.shape)
print("Shape YTrain -", ytrain.shape)
print("M -", len(xtrain))

Shape XTrain - (10, 3)
Shape YTrain - (10,)
M - 10


In [7]:
def entropy_function(y) :
    entropy = 0
    if len(y) != 0 :
        p1 = len(y[y==1])/len(y)

        if p1 != 1 and p1 != 0 :
            entropy = (-p1)*(np.log2(p1)) - ((1-p1)*(np.log2(1-p1)))
            
    return entropy

In [8]:
def split_function(X, nodeIndices, feature) :
    lIndices = []
    rIindices = []

    for i in nodeIndices :
        if X[i][feature] == 1 :
            lIndices.append(i)
        else :
            rIindices.append(i)
            
    return lIndices, rIindices


In [9]:
# Testing split_function

example_indices = [0,1,2,3,4,5,6,7,8,9]
example_feature = 0
lIndices, rIndices = split_function(xtrain, example_indices, example_feature)

print("Feature being 0, we are talking about the first row in the dataset. so it will segregate on that basis.")
print("left indices: ", lIndices)
print("right indices: ", rIndices)


Feature being 0, we are talking about the first row in the dataset. so it will segregate on that basis.
left indices:  [0, 1, 2, 3, 4, 7, 9]
right indices:  [5, 6, 8]


In [10]:
def info_gain_function(X, y, nodeIndices, feature) :
    
    lIndices, rIndices = split_function(X, nodeIndices, feature)

    Xnode, ynode = X[nodeIndices], y[nodeIndices]
    Xleft, yleft = X[lIndices], y[lIndices]
    Xright, yright = X[rIndices], y[rIndices]

    info_gain = 0

    nodeEntropy = entropy_function(ynode)
    leftEntropy = entropy_function(yleft)
    rightEntropy = entropy_function(yright)

    wleft = len(Xleft)/len(Xnode)
    wright = len(Xright)/len(Xnode)

    weighted_entropy = ((wleft*leftEntropy) + (wright*rightEntropy))
    info_gain = nodeEntropy - weighted_entropy
    
    return info_gain

In [11]:
print(type(info_gain_function))

<class 'function'>


In [12]:
# testing info_gain_function

example_info_gain = info_gain_function(xtrain, ytrain, example_indices, feature = 1)
print(example_info_gain)

0.12451124978365313


In [13]:
def best_data_split_function(X, y, nodeIndices) :
    numFeatures = X.shape[1]    # X[0 to 2] = 3
    bestFeature = -1
    max_info_gain = 0

    for i in range(numFeatures) :
        info_gain = info_gain_function(X, y, nodeIndices, i)

        if (info_gain > max_info_gain) :
            max_info_gain = info_gain
            bestFeature = i
    
    return bestFeature

In [14]:
main_indices = [0,1,2,3,4,5,6,7,8,9]
best_feature = best_data_split_function(xtrain, ytrain, main_indices)
print("Best feature = ", best_feature)

Best feature =  2


In [15]:
tree = []

def tree_building_function(X, y, nodeIndices, branchName, maxDepth, currDepth) :
    if currDepth == maxDepth :
        formatting = (" "*currDepth) + ("-"*currDepth)
        print(formatting, "%s leaf node with indices" % (branchName), nodeIndices)
        return
    
    best_feature = best_data_split_function(X, y, nodeIndices)

    formatting = ("-"*currDepth)
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, currDepth, branchName, best_feature))

    lIndices, rIndices = split_function(X, nodeIndices, best_feature)
    tree.append((lIndices, rIndices, best_feature))

    tree_building_function(X, y, lIndices, "Left", maxDepth, currDepth+1)
    tree_building_function(X, y, rIndices, "Right", maxDepth, currDepth+1)

In [16]:
tree_building_function(xtrain, ytrain, main_indices, "Root", maxDepth=2, currDepth=0)

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [0, 1, 4, 7]
  -- Right leaf node with indices [5]
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [8]
  -- Right leaf node with indices [2, 3, 6, 9]
